In [10]:
from tensorflow import keras
# caption_model = keras.models.load_model("./results/5000-model.caption", compile=False)
image_model = keras.models.load_model("./results/3000-model.image", compile=False)

In [11]:
import numpy as np
caption_representations = np.load('./caption_representations/3000.npy')
# image_representations = np.load('image-representations.npy')

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
resnet_model = ResNet50(weights='imagenet', include_top=False)

def extract_features(img_path): # though feature vector for training images has been precalculated and present in a file still function is needed for unseen images
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = resnet_model.predict(x)
  return np.expand_dims(features.flatten(), axis=0)

In [3]:
texts = []
with open('./test_labels/test_label_999.txt') as fp:
    for line in fp:
        tokens = line.strip().split()
        texts.append(' '.join(tokens[1:]))

In [12]:
l = open('recall_measure.txt', 'w')

In [13]:
def generate_caption(image_filename, img_caption, n=10):
  # generate image representation for new image
  image_representation = image_model.predict(extract_features(image_filename))
  # compute score of all captions in the dataset
  scores = np.dot(caption_representations, image_representation.T).flatten()
  # compute indices of n best captions
  indices = np.argpartition(scores, -n)[-n:]
  indices = indices[np.argsort(scores[indices])]
  # display them
  sum = 0
  for i in [int(x) for x in reversed(indices)]:
    print(scores[i], texts[i])
    l.write(texts[i] + '\n')
    if(texts[i] == img_caption):
        sum = sum + 1
  return sum

In [14]:
k = 10 # Recall@K
count = 0
num = 1000
sum_recall = 0
img_path = '../Recipe1M/recipe1M_images_test/test/'
with open('./test_labels/test_label_999.txt') as f:
    while(True):
        if(count >= num):
            break
        line = f.readline()
        img_name = line.split(' ')[0]
        # print(img_name)
        img_caption = []
        tokens = line.strip().split()
        img_caption.append(' '.join(tokens[1:]))
        img_caption = img_caption[0]
        print(img_caption)
        l.write(img_caption + '\n')

        img_title = img_name.split('.')[0]
        # print(img_title)

        img_path_final = img_path
        for i in range(4):
            img_path_final = img_path_final + img_title[i] + '/'
        img_path_final = img_path_final + img_name
        sum_recall = sum_recall + generate_caption(img_path_final, img_caption)
        # print('cp ' + img_path_final + ' V2-20000')
        # os.system('cp ' + img_path_final + ' V2-20000')
        count = count + 1
        print(count)
        l.write(str(count) + '\n')
print(sum_recall)

epherd Pie
4.6201572 Vegetarian Shepherds Pie
4.562152 Mustardy Potato Pierogies
4.5482244 Baked Meatball Ziti
4.2985134 Chicken Baked Ricotta Manicotti
4.212268 Cucumbers in Sour Cream (Low Fat or Non Fat)
4.1047826 Italian Vegetarian Patties
4.017642 Good Ole Chicken Creole
4.0117207 A Nice Slow-Cooked Pork
4.010575 Turkey Picadillo
954
Lemon Chickpeas
5.0624714 Mediterranean Sausage Chili #Ragu
4.441465 Wild-mushroom and Potato Stew
4.365615 Sauteed Green Beans
4.269445 Kalops (Swedish Beef Stew)
4.2251577 Spanish Caldo Gallego
4.196766 Hibachi style Japanese rice
4.1676817 Ultimate Black Bean and Rice Soup
4.1287093 Vintage Hamburger Goulash
4.107173 Vietnamese Grilled Pork over Vermicelli Noodles
4.072852 Puertorican RED Rice & Beans
955
Homemade Heart Shaped Cookies
6.162368 Popover Egg Pies
6.1085825 Crusty Yummy Biscuits
6.0839763 Sticky Gingerbread
6.066633 Steamed Pork Buns
6.0254173 Guyanese Coconut Buns
6.004732 White Butter Cake layered with Raspberry Cream with a White Ch

In [15]:
l.close()